In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd


In [3]:
anime_data = pd.read_csv('../input/anime-recommendations-database/anime.csv')
anime_data.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
# getting shape 
anime_data.shape

(12294, 7)

In [5]:
anime_data.dtypes

anime_id      int64
name         object
genre        object
type         object
episodes     object
rating      float64
members       int64
dtype: object

In [12]:
print(anime_data.describe().to_markdown())

|       |   anime_id |      rating |         members |
|:------|-----------:|------------:|----------------:|
| count |   12294    | 12064       | 12294           |
| mean  |   14058.2  |     6.4739  | 18071.3         |
| std   |   11455.3  |     1.02675 | 54820.7         |
| min   |       1    |     1.67    |     5           |
| 25%   |    3484.25 |     5.88    |   225           |
| 50%   |   10260.5  |     6.57    |  1550           |
| 75%   |   24794.5  |     7.18    |  9437           |
| max   |   34527    |    10       |     1.01392e+06 |


In [15]:
# how much is the missing data

print(anime_data.isnull().sum().to_markdown())

|          |   0 |
|:---------|----:|
| anime_id |   0 |
| name     |   0 |
| genre    |  62 |
| type     |  25 |
| episodes |   0 |
| rating   | 230 |
| members  |   0 |
